In [1]:
import os
# from code.get_args import get_parser
from get_args import get_parser
# import get_args
from model import TermModel
import json
import torch
import torch.nn as nn
from transformers import BertTokenizerFast
from dataset import *
from train import *
import torch.utils.data as td
from samp import *
from dataset import *

parser = get_parser()
class Args:
    train_path = '/data2/private/wzt/all_train_addc_strict.jsonl'

args = Args()

def neg_ratio_static1(step):
    return 2

def neg_ratio_static2(step):
    return 1

def neg_ratio_dynamic1(step):
    return min(max(1, int(3.5 - step/10000)), 3)

def neg_ratio_dynamic2(step):
    return min(max(1, int(0.5 + step/10000)), 3)

def neg_ratio_static3(step):
    return 4

def neg_ratio_static4(step):
    return 5

def neg_ratio_dynamic3(step):
    if step < 15000:
        return 5
    elif step <25000:
        return 1
    else:
        return 3

def neg_ratio_dynamic4(step):
    if step < 15000:
        return 5
    else:
        return 3

sample_probs_funcs = [wzt_sample1, wzt_sample2, wzt_sample3, wzt_sample1, wzt_sample4]
neg_ratio_funcs = [neg_ratio_static1, neg_ratio_static2, neg_ratio_dynamic1, neg_ratio_dynamic2, neg_ratio_static3, neg_ratio_static4, neg_ratio_dynamic3, neg_ratio_dynamic4]



train_data = []
with open(args.train_path, 'r', encoding='utf-8') as f:
    for line in tqdm(f,desc='read train data'):
        train_data.append(json.loads(line))


read train data: 9558100it [02:33, 62414.23it/s] 


In [2]:
from testsample import *
sam = sample_probs_funcs[1]()
print(sam)
testsamp = Sample(sam)
total_negs = []
total_poss = []
from tqdm import tqdm
import traceback
try:
    for item in tqdm(train_data):
        if not item[0]:
            continue
        sent = 'b'+item[0]
        label = [(a+1,b+1) for a,b,c in item[1]]
        for a,b,c in item[1]:
            total_poss.append(c)
        cneg= [(b+1,c+1) for a,b,c in item[2]]
        neg = testsamp.get_negative(item[0],label,len(sent),cneg)
        for key,jtem in neg:
            total_negs.append((key,sent[jtem[0]:jtem[1]+1]))

except:
    traceback.print_exc()

    # break

{'random': 0.2, 'common': 0.8}


100%|██████████| 9558100/9558100 [04:08<00:00, 38530.36it/s]


In [3]:
# counter the total_negs[:,1]
from collections import Counter



neg_counter = Counter(total_negs)
json.dump(list(neg_counter.items()),open('neg_counter.json','w',encoding='utf-8'),ensure_ascii=False)
pos_counter = Counter(total_poss)
json.dump(list(pos_counter.items()),open('pos_counter.json','w',encoding='utf-8'),ensure_ascii=False)

In [7]:
tar = '男主角'
print(tar in neg_counter)
print(tar in pos_counter)
for item in tqdm(train_data):
    if not item[0]:
        continue
    sent = item[0]
    label = [c for a,b,c in item[1]]
    if tar in label:
        print(sent)
        print(item[1])

False
True


  3%|▎         | 306138/9558100 [00:00<00:11, 775813.42it/s]

美源发采（bigen）是hoyu的染发品牌，在香港的电视广告播放超过30年，属长寿广告之一，广告男主角是当年的粤语片小生曾江。
[[0, 3, '美源发采'], [17, 18, '染发'], [19, 20, '品牌'], [23, 24, '香港'], [26, 29, '电视广告'], [41, 42, '广告'], [46, 47, '广告'], [48, 50, '男主角'], [55, 57, '粤语片'], [60, 61, '曾江']]


  7%|▋         | 685348/9558100 [00:00<00:12, 722235.15it/s]

后在新加坡应征刘贝锦自制影片公司的默片演员，立即被录取在《新客》片中担任男主角，改艺名为“超人”，但此电影票房欠佳，该制片公司在1927年5月10日宣告结束营运。
[[2, 4, '新加坡'], [7, 9, '刘贝锦'], [17, 18, '默片'], [29, 30, '新客'], [36, 38, '男主角'], [51, 54, '电影票房'], [59, 62, '制片公司'], [64, 68, '1927年'], [69, 73, '5月10日'], [74, 77, '宣告结束']]


...

 90%|█████████ | 8638020/9558100 [00:10<00:01, 748412.79it/s]

《美国佳人》赢得了包括最佳影片、导演、男主角奖在内的五项大奖。
[[11, 14, '最佳影片'], [16, 17, '导演'], [19, 21, '男主角']]
执导《美国佳人》的山姆·曼德斯成为第6位凭处女作拿下导演奖的导演，而拿下男主角奖的凯文·斯派西则成为第一位首次提名男主角和男配角奖就成功胜出的男演员，他曾因《普通的嫌疑犯》获第68届奥斯卡金像奖男配角奖。
[[9, 14, '山姆·曼德斯'], [26, 27, '导演'], [30, 31, '导演'], [36, 39, '男主角奖'], [41, 46, '凯文·斯派西'], [57, 59, '男主角'], [61, 64, '男配角奖'], [71, 73, '男演员'], [87, 96, '第68届奥斯卡金像奖'], [97, 100, '男配角奖']]
此外，西恩·潘和蒂姆·罗宾斯分别因《神秘河流》拿下男主角和男配角奖，该片也成为历史上第四部获得两个男演员表演奖项的电影。
[[3, 6, '西恩·潘'], [8, 13, '蒂姆·罗宾斯'], [25, 27, '男主角'], [29, 32, '男配角奖'], [49, 51, '男演员']]


 93%|█████████▎| 8936566/9558100 [00:11<00:00, 732765.45it/s]

男主角奖得主杰米·福克斯成为历史上第10位在同一年中获两项表演类奖项提名的演员，也是第2位达成这一目标的男演员。
[[0, 2, '男主角'], [6, 11, '杰米·福克斯'], [17, 20, '第10位']]


100%|██████████| 9558100/9558100 [00:12<00:00, 796282.38it/s]


In [5]:
pos_counter['男主角']

50

In [ ]:
for k,v in neg_counter.items():
    # print(k,v)
    if '浓硫酸' in k:
        print(k,v)

In [ ]:
def read_term_tables(path,retset = True):
    res = []
    with open(path,'r') as f:
        for line in f:
            res.append(line.strip())
        # res = f.readlines()
    
    return set(res) if retset else res
def have_addroot(s,add_bigrams):
    for x in add_bigrams:
        if x in s:
            return True
    return False
add_bigrams = read_term_tables('/data2/private/wzt/term10_18/term/term_tables/add_bigrams.txt')
all_bigrams = read_term_tables('/data2/private/wzt/term10_18/term/term_tables/all_bigrams.txt',retset=False)
all_unigrams = read_term_tables('/data2/private/wzt/term10_18/term/term_tables/all_unigrams.txt')
checkpunc = read_term_tables('/data2/private/wzt/term10_18/term/term_tables/checkpunc.txt')
shumings = read_term_tables('/data2/private/wzt/term10_18/term/term_tables/shumings.txt')
titles = read_term_tables('/data2/private/wzt/term10_18/term/term_tables/titles.txt')
common_grams = read_term_tables('/data2/private/wzt/term10_18/term/term_tables/common_grams.txt')
high_bigrams = read_term_tables('/data2/private/wzt/term10_18/term/term_tables/high_bigrams.txt')
add_common = read_term_tables('/data2/private/wzt/term10_18/term/term_tables/add_common.txt')
common_grams = common_grams.union(add_common)

In [ ]:
# load pos_counter.json
import json
pos_counter = json.load(open('pos_counter.json','r',encoding='utf-8'))

In [ ]:
pos_counter23 = [ (k,v) for k,v in pos_counter if len(k)>=2 and len(k)<=3]
pos_counter23 = sorted(pos_counter23,key=lambda x:x[1],reverse=True)
pos_counter23

In [ ]:
# save pos_counter23 to lines
with open('pos_counter23.txt','w',encoding='utf-8') as f:
    for k,v in pos_counter23:
        f.write(k+'\t'+str(v)+'\n')

In [ ]:
posset = set([k for k,v in pos_counter])
newc = common_grams - posset
len(newc)

In [ ]:
# save newc to lines
with open('newc.txt','w',encoding='utf-8') as f:
    for k in newc:
        f.write(k+'\n')

In [ ]:
inter = common_grams.intersection(posset)
# '稀盐酸' in inter
# print(inter)
# save inter
with open('inter.txt','w',encoding='utf-8') as f:
    for k in inter:
        f.write(k+'\n')

In [ ]:
# load pos_counter.json
import json
pos_counter = json.load(open('pos_counter.json','r',encoding='utf-8'))
# load neg
import json
neg_counter = json.load(open('neg_counter.json','r',encoding='utf-8'))

In [ ]:
tar = '其他类型'
print(tar in pos_counter)
print(tar in neg_counter)

In [ ]:
neg_counter

In [ ]:
posr = []
negr = []
tar = '其他'
from tqdm import tqdm
for k,v in tqdm(pos_counter):
    if tar in k:
        posr.append((k,v))
for k,v in tqdm(neg_counter):
    if k.startswith(tar):
        negr.append((k,v))


In [ ]:
from string import punctuation as enpunc
from zhon.hanzi import punctuation as zhpunc
punc = enpunc + zhpunc
def have_punc(s,punc):
    s = set(s)
    for x in punc:
        if x in s:
            return True
    return False

In [ ]:
puncpos = []
for item in tqdm(pos_counter):
    if have_punc(item[0],punc):
        puncpos.append(item)

In [31]:
# puncpos
checkpunc = '：，！!:？>。<#&～_…?%'
def havecheck(s,checkpunc):
    for x in checkpunc:
        if x in s:
            return True
    if s[0]=='、' or s[-1]=='、':
        return True
    return False
from collections import Counter
def have_single_pair(s):
    # s = set(s)
    s = Counter(s)
    pairs = ['《》','“”','’‘']
    for p in pairs:
        if p[0] in s and p[1] not in s:
            return True
        if p[1] in s and p[0] not in s:
            return True
    if s['\'']==1 or s['\"']==1:
        return True
    return False

puncins = []
puncouts = []
for item in puncpos:
    if havecheck(item[0],checkpunc) or have_single_pair(item[0]):
        puncouts.append(item)
    else:
        puncins.append(item)

# tmp  = [item for item in puncpos if havecheck(item[0],checkpunc) or have_single_pair(item[0])]
# tmp = sorted(tmp,key=lambda x:x[1],reverse=True)

tmp = [item for item in puncins if  have_punc(item[0][0],punc) or have_punc(item[0][-1],punc)]
tmp = sorted(tmp,key=lambda x:x[1],reverse=True)
# # tmp = [item for item in tmp if not havecheck(item[0],checkpunc)]
# # tmp = [item for item in tmp if item[0][0]!='《' and item[0][0]!='“' and item[0][0]!='\"']
# # tmp = [item for item in tmp if have_single_pair(item[0])]
tmp

[['《圣经》', 875],
 ['《古兰经》', 859],
 ['《纽约时报》', 778],
 ['《人民日报》', 745],
 ['《红楼梦》', 576],
 ['《西游记》', 346],
 ['《左传》', 305],
 ['《诗经》', 293],
 ['《福布斯》', 289],
 ['《告示牌》', 280],
 ['《水浒传》', 253],
 ['《中国时报》', 253],
 ['《晚间新闻》', 225],
 ['《迷失》', 221],
 ['《卫报》', 211],
 ['《四库全书》', 191],
 ['《科学》', 188],
 ['台湾《壹周刊》', 186],
 ['《华盛顿邮报》', 184],
 ['“大和”', 182],
 ['《易经》', 181],
 ['《环球时报》', 181],
 ['《大学》', 180],
 ['《告示牌》百强单曲榜', 179],
 ['《南方周末》', 173],
 ['《壹周刊》', 173],
 ['《周礼》', 164],
 ['《时代》', 157],
 ['阿道夫·', 157],
 ['《水经注》', 155],
 ['《新青年》', 148],
 ['《华尔街日报》', 147],
 ['《泰晤士报》', 147],
 ['《时代》杂志', 145],
 ['《超级星光大道》', 141],
 ['《孟子》', 138],
 ['《广韵》', 137],
 ['“中华民国”', 134],
 ['《纽约客》', 133],
 ['《道德经》', 132],
 ['《光明日报》', 129],
 ['《真理报》', 125],
 ['《创世纪》', 125],
 ['《南方都市报》', 121],
 ['《历史》', 119],
 ['《中庸》', 111],
 ['《华严经》', 109],
 ['《滚石》杂志', 109],
 ['《生活》', 106],
 ['《红旗》杂志', 105],
 ['《英雄联盟》', 104],
 ['《金融时报》', 104],
 ['《晨报》', 101],
 ['《本草纲目》', 101],
 ['《马太福音》', 99],
 ['“联盟”', 97],
 ['《自然》', 95],
 ['《尤利西斯》', 88],
 ['《

In [27]:
tmp = sorted(tmp,key=lambda x:x[1],reverse=True)
tmp

[['比利时、', 442],
 ['资治通鉴》', 295],
 ['东南亚、', 254],
 ['主题曲“', 225],
 ['台湾《壹周刊》', 186],
 ['山海经》', 164],
 ['阿道夫·', 157],
 ['、维也纳', 90],
 ['洛杉矶时报》', 70],
 ['阿塞拜疆、', 69],
 ['计算机科学、', 63],
 ['城市论坛》', 62],
 ['约翰福音》', 62],
 ['美女与野兽》', 61],
 ['反“扫荡”', 60],
 ['互联网+', 60],
 ['台湾民报》', 59],
 ['半导体、', 57],
 ['录音室专辑、', 52],
 ['、三民路', 52],
 ['、发明家', 46],
 ['搜神记》', 39],
 ['民主300+', 39],
 ['5pb.', 37],
 ['新马路、', 36],
 ['狮子山下》', 35],
 ['香港2030+', 32],
 ['等等...', 31],
 ['、新喀里多尼亚', 29],
 ['反“围剿”', 28],
 ['大金刚》', 27],
 ['迪士尼+', 27],
 ['台湾《苹果日报》', 26],
 ['平面设计、', 25],
 ['吸血鬼日记》', 23],
 ['巴厘岛、', 22],
 ['太空堡垒卡拉狄加》', 22],
 ['英国《卫报》', 21],
 ['中西区、', 21],
 ['药剂师、', 20],
 ['香港《大公报》', 20],
 ['·奥尔良', 20],
 ['周刊新潮》', 20],
 ['塔吉克、', 19],
 ['〈世界名作剧场〉', 18],
 ['公视+', 18],
 ['、男演员', 17],
 ['修订《逃犯条例》', 17],
 ['牧场物语》', 16],
 ['哈尔滨工业大学（', 16],
 ['脱衣舞娘》', 16],
 ['红磡站）', 15],
 ['电影《红楼梦》', 15],
 ['伪装者》', 14],
 ['飘忽s-', 14],
 ['暗黑龙与光之剑》', 14],
 ['w-inds.', 13],
 ['破“四旧”', 12],
 ['feel.', 12],
 ['预警机、', 12],
 ['剪刀手爱德华》', 12],
 ['全

In [ ]:
posr

In [ ]:
len(pos_counter)